### 1. 基本概念
1.定义
- LangChain中的检索式问答链
- 结合文档检索和LLM的问答系统
- 支持上下文增强的回答生成
2. 核心组件

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma

# 基本初始化
retrieval_qa = RetrievalQA.from_chain_type(
    llm=Ollama(model="llama2"),
    chain_type="stuff",  # 链类型
    retriever=vectorstore.as_retriever(),  # 检索器
    return_source_documents=True  # 返回源文档
)

1. 基本作用
from_chain_type是RetrievalQA的工厂方法，用于创建不同类型的检索问答链。
2. 参数详解

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    # 必需参数
    llm=llm,                    # 语言模型实例
    chain_type="stuff",         # 链类型
    retriever=retriever,        # 检索器实例

    # 可选参数
    return_source_documents=True,  # 是否返回源文档
    chain_type_kwargs={           # 链类型特定参数
        "prompt": custom_prompt,
        "verbose": True
    },
    verbose=True,                # 是否显示详细日志
)

chain_type
- "stuff": 直接将所有文档合并，适合处理少量文档
- "map_reduce": 分步处理大量文档
- "refine": 逐步细化答案，适合需要高质量答案的场景
- "map_rerank": 对答案进行重新排序

### 2. 工作原理
1. 检索过程
2. 问答生成

In [2]:
# 检索流程示例
class CustomRetrievalQA:
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever
    
    def process_query(self, query):
        # 1. 检索相关文档
        relevant_docs = self.retriever.get_relevant_documents(query)
        
        # 2. 构建上下文
        context = "\n".join([doc.page_content for doc in relevant_docs])
        
        # 3. 构建提示词
        prompt = f"""
        基于以下信息回答问题：
        
        {context}
        
        问题：{query}
        """
        
        # 4. 生成回答
        return self.llm.predict(prompt)

### 3. 高级配置
1.链类型选择

In [ ]:
# 不同链类型的配置
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 直接将所有文档合并
    retriever=retriever
)

qa_map_reduce = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",  # 分步处理大量文档
    retriever=retriever
)

qa_refine = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # 逐步细化答案
    retriever=retriever
)

2.检索器配置

In [ ]:
# 自定义检索器配置
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 最大边际相关性搜索
    search_kwargs={
        "k": 3,  # 返回文档数量
        "fetch_k": 10,  # 预筛选数量
        "lambda_mult": 0.7  # 多样性权重
    }
)

### 4. 实际应用示例
1. 知识库问答系统


In [ ]:
class KnowledgeBaseQA:
    def __init__(self, documents):
        # 初始化嵌入模型
        self.embeddings = OllamaEmbeddings(model="bge-m3")
        
        # 创建向量存储
        self.vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings
        )
        
        # 初始化LLM
        self.llm = Ollama(model="llama2")
        
        # 创建QA链
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(),
            return_source_documents=True
        )
    
    def ask(self, question):
        result = self.qa_chain.invoke({"query": question})
        return {
            "answer": result["result"],
            "sources": [doc.page_content for doc in result["source_documents"]]
        }

2. 多语言问答系统


In [ ]:
class MultilingualQA:
    def __init__(self):
        self.qa_chains = {}
        
    def add_language(self, lang, documents):
        # 为每种语言创建独立的QA链
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=OllamaEmbeddings(model="bge-m3")
        )
        
        self.qa_chains[lang] = RetrievalQA.from_chain_type(
            llm=Ollama(model="llama2"),
            chain_type="stuff",
            retriever=vectorstore.as_retriever()
        )
    
    def ask(self, question, lang):
        if lang not in self.qa_chains:
            raise ValueError(f"Language {lang} not supported")
        
        return self.qa_chains[lang].invoke({"query": question})

### 5. 性能优化
1. 缓存机制

In [ ]:
from functools import lru_cache

class CachedQA:
    def __init__(self, qa_chain):
        self.qa_chain = qa_chain
    
    @lru_cache(maxsize=1000)
    def get_answer(self, question):
        return self.qa_chain.invoke({"query": question})

2. 批处理优化


In [ ]:
class BatchQA:
    def __init__(self, qa_chain):
        self.qa_chain = qa_chain
    
    async def process_batch(self, questions, batch_size=5):
        results = []
        for i in range(0, len(questions), batch_size):
            batch = questions[i:i + batch_size]
            tasks = [
                self.qa_chain.ainvoke({"query": q})
                for q in batch
            ]
            batch_results = await asyncio.gather(*tasks)
            results.extend(batch_results)
        return results

### 6. 最佳实践
1. 错误处理


In [ ]:
class RobustQA:
    def __init__(self, qa_chain):
        self.qa_chain = qa_chain
    
    def safe_ask(self, question):
        try:
            result = self.qa_chain.invoke({"query": question})
            return {
                "status": "success",
                "answer": result["result"],
                "sources": result.get("source_documents", [])
            }
        except Exception as e:
            return {
                "status": "error",
                "message": str(e),
                "fallback_answer": "抱歉，我现在无法回答这个问题。"
            }

2. 结果验证


In [ ]:
def validate_answer(result):
    # 检查答案质量
    answer = result["result"]
    sources = result["source_documents"]
    
    # 验证答案长度
    if len(answer) < 10:
        return False, "答案过短"
    
    # 验证源文档支持
    if not sources:
        return False, "没有找到相关文档支持"
    
    return True, "验证通过"

RetrievalQA是构建智能问答系统的强大工具，它能够：
- 结合文档检索和LLM能力
- 提供基于上下文的准确回答
- 支持多种处理策略
- 适应不同规模的应用需求
  
通过合理配置和优化，RetrievalQA可以帮助我们构建高质量的问答系统。